# Farina in Natura - estratégia

![](Logo_farina_in_natura_edited.jpg)

## Importando os dados e observações iniciais

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import nbformat
import chart_studio
import plotly.express as px
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected = True )
cf.go_offline()
# chart_studio.tools.set_credentials_file(username='claudio.oliveira', api_key='fXrLqCmkdJQyTheCzYMS')
from IPython.display import display


In [2]:
from fpdf import FPDF

In [3]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [5]:
transactions.head()

,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,46.0,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [6]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   item        25922 non-null  float64
 5   produto     25992 non-null  object 
 6   quantidade  25992 non-null  float64
 7   valor_unit  25992 non-null  object 
 8   desconto    25992 non-null  float64
 9   total_item  25992 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.0+ MB


# Limpeza e manipulação dos dados

In [7]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

venda            6
data_hora        6
cliente          6
mesa          3219
item            76
produto          6
quantidade       6
valor_unit       6
desconto         6
total_item       6
dtype: int64

In [8]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [9]:
# Delete the column "item"

del transactions["item"]

In [10]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   produto     25992 non-null  object 
 5   quantidade  25992 non-null  float64
 6   valor_unit  25992 non-null  object 
 7   desconto    25992 non-null  float64
 8   total_item  25992 non-null  object 
dtypes: float64(3), object(6)
memory usage: 2.0+ MB


,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [11]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [12]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
121,520.0,02/02/2022 16:57,CONSUMIDOR,M0,BOMBOLONI NUTELLA 80G,2.0,12,0.0,24
122,520.0,02/02/2022 16:57,CONSUMIDOR,M0,TARTELETE DE MORANGO 120G,1.0,18,0.0,18
123,520.0,02/02/2022 16:57,CONSUMIDOR,M0,CROISSANT MUSSATRELA DE,1.0,"25,5",0.0,"25,5"
894,856.0,11/02/2022 19:19,CONSUMIDOR,M0,CROISSANT BRIE PARMA MEL DE,2.0,"25,5",0.0,51
895,856.0,11/02/2022 19:19,CONSUMIDOR,M0,VH SANTA HORTENSIA SAUVIG,1.0,59,0.0,59


In [13]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [14]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [15]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [16]:
# change the decimal delimiter of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [17]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "total_item" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)

In [18]:


# Convert "data_hora" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

# Creating a horario column that is a copy of data_hora and coverting it to datetime and extracting only the day;
transactions['horario'] = (transactions['data_hora'])
transactions['horario'] = transactions['horario'].dt.date

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [19]:
transactions.head()

,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,horario
0,460,2022-02-01 08:13:00,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01
1,460,2022-02-01 08:13:00,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01
2,461,2022-02-01 08:14:00,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01
3,461,2022-02-01 08:14:00,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01
4,465,2022-02-01 09:01:00,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,2022-02-01


In [20]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   venda       25992 non-null  int64         
 1   data_hora   25992 non-null  datetime64[ns]
 2   cliente     25992 non-null  object        
 3   mesa        25992 non-null  object        
 4   produto     25992 non-null  object        
 5   quantidade  25992 non-null  int64         
 6   valor_unit  25992 non-null  float64       
 7   desconto    25992 non-null  float64       
 8   total_item  25992 non-null  float64       
 9   horario     25992 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 2.2+ MB


In [21]:

# https://www.dataquest.io/blog/python-datetime-tutorial/


#  add hora and minuto column; using class attributes .hour and .minute

transactions['hora'] = transactions.data_hora.dt.hour
transactions['minuto'] = transactions.data_hora.dt.minute



### Renaming the column "data_hora" to "data"

In [22]:
# https://www.statology.org/pandas-rename-columns/
# renaming the column

transactions.rename(columns = {'data_hora':'data'}, inplace = True)


### Extracting only the Date from 'data' column

In [23]:

## https://datagy.io/pandas-extract-date-from-datetime/
transactions['data'] = transactions['data'].dt.normalize()

In [24]:
# Definindo a função time_of_day
def time_of_day(hora):
    """
    Determina se a  compra foi feita pela manhã, tarde ou noite.
    """
    
    if hora < 12:
        
        return 'manhã'
    
    elif hora < 17:
        
        return 'tarde'
    
    else:
        
        return 'noite'

In [25]:
# Apply time of day function to 'hora' column

transactions['time_of_day'] = transactions.hora.apply(time_of_day)


In [26]:
# https://pt.stackoverflow.com/questions/508484/mostrar-dias-da-semana#:~:text=Como%20weekday()%20retorna%20um,fim%20de%20semana%20ou%20n%C3%A3o.

import locale
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [27]:
# Add a day of week variable and a second classifier of weekday or weekend

from datetime import date
import calendar

transactions['day_of_week'] = [calendar.day_name[i.weekday()] for i in transactions['horario']]


In [28]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,horario,hora,minuto,time_of_day,day_of_week
0,460,2022-02-01,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,13,manhã,terça-feira
1,460,2022-02-01,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,13,manhã,terça-feira
2,461,2022-02-01,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,14,manhã,terça-feira
3,461,2022-02-01,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,14,manhã,terça-feira
4,465,2022-02-01,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,2022-02-01,9,1,manhã,terça-feira


In [29]:

transactions['day_type'] = ['FDS' if (i == 'domingo') | (i == 'sábado') else 'DDS' for i in 
                                                                            transactions['day_of_week']]


In [30]:
transactions.rename(columns = {'day_of_week':'dia_da_semana'}, inplace = True)

In [31]:
transactions.rename(columns = {'time_of_day':'hora_do_dia'}, inplace = True)

In [32]:
transactions.rename(columns = {'day_type':'tipo_de_dia'}, inplace = True)

In [33]:
# Add an item count variable equal to one for future groupby operations

transactions['item_count'] = 1


In [34]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,horario,hora,minuto,hora_do_dia,dia_da_semana,tipo_de_dia,item_count
0,460,2022-02-01,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,13,manhã,terça-feira,DDS,1
1,460,2022-02-01,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,13,manhã,terça-feira,DDS,1
2,461,2022-02-01,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,14,manhã,terça-feira,DDS,1
3,461,2022-02-01,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,14,manhã,terça-feira,DDS,1
4,465,2022-02-01,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,2022-02-01,9,1,manhã,terça-feira,DDS,1


In [35]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   venda          25992 non-null  int64         
 1   data           25992 non-null  datetime64[ns]
 2   cliente        25992 non-null  object        
 3   mesa           25992 non-null  object        
 4   produto        25992 non-null  object        
 5   quantidade     25992 non-null  int64         
 6   valor_unit     25992 non-null  float64       
 7   desconto       25992 non-null  float64       
 8   total_item     25992 non-null  float64       
 9   horario        25992 non-null  object        
 10  hora           25992 non-null  int64         
 11  minuto         25992 non-null  int64         
 12  hora_do_dia    25992 non-null  object        
 13  dia_da_semana  25992 non-null  object        
 14  tipo_de_dia    25992 non-null  object        
 15  item_count     2599

# Verificando alguma divergência dos dados

In [36]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [37]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)


In [38]:
# continuando o item anterior
resumo = transactions.loc[transactions["confirm"] == True].sum()
print (resumo)

venda                                                        13730391
cliente             CONSUMIDORCONSUMIDORCONSUMIDORCONSUMIDORIFOODC...
mesa                M 5M 13M 11M 5NDM 19NDM 13M 13M 17NDM 23M 23M ...
produto             CROISSANTCROISSANTCROISSANTCROISSANTCROISSANT ...
quantidade                                                       2139
valor_unit                                                    21083.7
desconto                                                          0.0
total_item                                                    53878.3
hora                                                            29332
minuto                                                          59296
hora_do_dia         manhãmanhãtardemanhãmanhãmanhãmanhãnoitenoiten...
dia_da_semana       sexta-feirasexta-feirasexta-feirasábadosábados...
tipo_de_dia         DDSDDSDDSFDSFDSFDSFDSFDSFDSFDSFDSFDSFDSFDSFDSF...
item_count                                                       2032
total_item_verif    

C:\Users\ctobr\AppData\Local\Temp\ipykernel_24552\2200306820.py:2: FutureWarning:

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



In [39]:
# Gerando um relatório em pdf dos dados divergentes
# How do I select specific rows and columns from a DataFrame?
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe
# https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/

ts2 = transactions.loc[transactions["confirm"] == True, ["venda", "produto", "quantidade", "valor_unit", "total_item", "total_item_verif"]]

In [40]:
# Exporting to a pdf file the transactions that has a inconsistent data.
from matplotlib.backends.backend_pdf import PdfPages


In [41]:
# https://levelup.gitconnected.com/how-to-write-a-pandas-dataframe-as-a-pdf-5cdf7d525488

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis("tight")
    ax.axis('off')
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    return fig


In [42]:
# https://www.alixaprodev.com/2022/05/convert-pandas-dataframe-to-pdf-in-python.html
#fig, ax =plt.subplots(figsize=(14,6))
#ax.axis('equal')
#ax.axis('off')
#the_table = ax.table(cellText=ts2.values,colLabels=ts2.columns,loc='center')




In [43]:
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [44]:
### Trying to Align the page number
# https://www.geeksforgeeks.org/matplotlib-figure-figure-text-in-python/
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            #if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                #fig.text(0.5, 0.5/pagesize[0],
                       # "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                       # verticalalignment ='bottom', 
                       # horizontalalignment ='right',
                       #transform = ax.transAxes,
                        #color ='green',
                        #fontsize = 5
                         #)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

### Creating a pdf file from the divergent data

In [45]:
# dataframe_to_pdf(ts2, "diverge_01.pdf", numpages=(12, 1), pagesize=(11.7, 8.3))

In [46]:
## Create a HTML file of the resumé "ts2" dataframe.

f = open('exp.html','w')
a = ts2.to_html()
f.write(a)
f.close()

In [47]:
# drop the 'confirm" and "total_item_verif" column
transactions = transactions.drop(['confirm'], axis=1)
transactions = transactions.drop(['total_item_verif'], axis=1)


In [48]:
transactions.head()

,venda,data,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item,horario,hora,minuto,hora_do_dia,dia_da_semana,tipo_de_dia,item_count
0,460,2022-02-01,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,13,manhã,terça-feira,DDS,1
1,460,2022-02-01,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,13,manhã,terça-feira,DDS,1
2,461,2022-02-01,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1,14.5,0.0,14.5,2022-02-01,8,14,manhã,terça-feira,DDS,1
3,461,2022-02-01,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1,15.5,0.0,15.5,2022-02-01,8,14,manhã,terça-feira,DDS,1
4,465,2022-02-01,CONSUMIDOR,M 1,QUICHE BACON 90G,2,19.5,0.0,39.0,2022-02-01,9,1,manhã,terça-feira,DDS,1


In [49]:
transactions.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   venda          25992 non-null  int64         
 1   data           25992 non-null  datetime64[ns]
 2   cliente        25992 non-null  object        
 3   mesa           25992 non-null  object        
 4   produto        25992 non-null  object        
 5   quantidade     25992 non-null  int64         
 6   valor_unit     25992 non-null  float64       
 7   desconto       25992 non-null  float64       
 8   total_item     25992 non-null  float64       
 9   horario        25992 non-null  object        
 10  hora           25992 non-null  int64         
 11  minuto         25992 non-null  int64         
 12  hora_do_dia    25992 non-null  object        
 13  dia_da_semana  25992 non-null  object        
 14  tipo_de_dia    25992 non-null  object        
 15  item_count     2599

> # **Exploração dos Dados**

In [50]:
# Find Unique Values of each item sold in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
transactions.produto.value_counts()

CROISSANT                         2393
PASTEL DE NATA 60G                1339
CROISSANT  AMENDOAS 100G           742
FOLHADO MACA 80G                   719
SANDUICHE DE PEPERONI BRIE         596
COOKIES 80G                        559
CROISSANT IFOOD                    534
SOURDOUGH AZEITONA E ALECRIM       517
FOLHADO FRANGO 160G                500
BOMBOLONI DOCE DE LEITE 80G        487
FOCACCIA FORMAGGIO 246G            460
AGUA MINERAL S/GAS 500ML           446
TORRADA FARINA 165G                435
CAFE EQUILIBRADO CAPSULA P         431
SOURDOUGH 369G                     431
LARANJA 400 ML                     430
MINI CROISSANT 25G KIT C 5         427
CROISSANT BRIE PARMA MEL DE        423
PIZZA AL TAGLIO GORGONZOLA E       421
MINI PAIN AU CHOCOLAT 35G          419
BOMBOLONI NUTELLA 80G              409
CROISSANT  TRADICIONAL 80 G        396
QUICHE DE FRANGO ALHO PORO         360
AGUA MINERAL SEM GAS 500 ML        341
COCA COLA LATA 350 ML              337
SUCO DE LARANJA 400ML    

In [51]:
# find the percentage each item is selling
transactions.produto.value_counts(True)

CROISSANT                         0.092067
PASTEL DE NATA 60G                0.051516
CROISSANT  AMENDOAS 100G          0.028547
FOLHADO MACA 80G                  0.027662
SANDUICHE DE PEPERONI BRIE        0.022930
COOKIES 80G                       0.021507
CROISSANT IFOOD                   0.020545
SOURDOUGH AZEITONA E ALECRIM      0.019891
FOLHADO FRANGO 160G               0.019237
BOMBOLONI DOCE DE LEITE 80G       0.018737
FOCACCIA FORMAGGIO 246G           0.017698
AGUA MINERAL S/GAS 500ML          0.017159
TORRADA FARINA 165G               0.016736
CAFE EQUILIBRADO CAPSULA P        0.016582
SOURDOUGH 369G                    0.016582
LARANJA 400 ML                    0.016544
MINI CROISSANT 25G KIT C 5        0.016428
CROISSANT BRIE PARMA MEL DE       0.016274
PIZZA AL TAGLIO GORGONZOLA E      0.016197
MINI PAIN AU CHOCOLAT 35G         0.016120
BOMBOLONI NUTELLA 80G             0.015736
CROISSANT  TRADICIONAL 80 G       0.015235
QUICHE DE FRANGO ALHO PORO        0.013850
AGUA MINERA

In [52]:
## Get unique names in the "produto" column using Series.unique() 
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
num_itens_vend = transactions.produto.unique().size

print('O número de diferentes itens vendido pela loja é:', num_itens_vend)

O número de diferentes itens vendido pela loja é: 212


In [53]:
# Look at the average number of items purchased...

avg_items = transactions.groupby(by='venda')['produto'].count().mean()

print('A média de itens por venda é: %.2f itens' %avg_items)

A média de itens por venda é: 2.79 itens


In [54]:
# ...and how that varies by time of day

single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['hora','minuto','hora_do_dia','data', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens vendidos por hora do dia é: \n{0}'.format(
    single_transaction.groupby(by='hora_do_dia')['item_count'].mean()))

O número médio de itens vendidos por hora do dia é: 
hora_do_dia
manhã    3.046419
noite    2.663510
tarde    2.783376
Name: item_count, dtype: float64


In [55]:
# Find the frequency of each item purchased and by time of day

item_frequencies_all = transactions.produto.value_counts()
item_frequencies_morning = transactions[transactions.hora_do_dia == 'manhã'].produto.value_counts()
item_frequencies_afternoon = transactions[transactions.hora_do_dia == 'tarde'].produto.value_counts()
item_frequencies_evening = transactions[transactions.hora_do_dia == 'noite'].produto.value_counts()

In [132]:
# Plot the frequency of purchase for each item

# https://community.plotly.com/t/errors-accessing-plotly-api/33271
# ok, so I found a solution to my problem - when I let my program run through, the error message produced is:
#chart_studio.exceptions.PlotlyRequestError: Authentication credentials were not provided.
#I Googled this error and there were suggestions to run plotly in offline mode by replacing:
#import chart_studio.plotly as py
#with

import plotly.offline as py
#then use py.plot instead of py.iplot

# Solving the problem of the 'plotly' online library with chart_studio
# https://plotly.com/python/getting-started-with-chart-studio/
import plotly.graph_objs as go
#import chart_studio.plotly as py
# import plotly.express as px

button_layer_1_height = 1.12
button_layer_2_height = 1.065

item_threshold_all = 100
item_threshold_other = 20

trace_all = go.Bar(
            x=item_frequencies_all[item_frequencies_all > item_threshold_all].index,
            y=item_frequencies_all[item_frequencies_all > item_threshold_all].values,
            visible=True,
            name='Todo dia'
    )

trace_morning = go.Bar(
            x=item_frequencies_morning[item_frequencies_morning > item_threshold_other].index,
            y=item_frequencies_morning[item_frequencies_morning > item_threshold_other].values,
            visible=False,
            name='Manhã'
    )

trace_afternoon = go.Bar(
            x=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].index,
            y=item_frequencies_afternoon[item_frequencies_afternoon > item_threshold_other].values,
            visible=False,
            name='Tarde'
    )

trace_evening = go.Bar(
            x=item_frequencies_evening[item_frequencies_evening > 10].index,
            y=item_frequencies_evening[item_frequencies_evening > 10].values,
            visible=False,
            name='Noite'
    )

data = [trace_all, trace_morning, trace_afternoon, trace_evening]

updatemenus = list([
    
    dict(
         buttons=list([   
            dict(label = 'Todo dia',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         {'title': 'Todo Dia'}]),
             
            dict(label = 'Manhã',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         {'title': 'Manhã'}]),
             
            dict(label = 'Tarde',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         {'title': 'Tarde'}]),
             
            dict(label = 'Noite',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         {'title': 'Noite'}])
        ]),
        type='buttons',
        direction = 'right',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.3,
        xanchor = 'left',
        y = button_layer_1_height,
        yanchor = 'top' )
])
                
layout = dict(title='Vendas Farina por hora do dia', showlegend=False,
              updatemenus=updatemenus)




fig = dict(data=data, layout=layout)
py.iplot(fig, filename='frequencies-bar.html')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



# Creating an animated frequencie line 

In [57]:
# Sales trend by date
# https://www.youtube.com/watch?v=Xjm7fOw6gjY
# use reset index for transforming the data in a variable not a index

grouped_by_date = pd.DataFrame(transactions.groupby(by='horario')['item_count'].sum().reset_index())

In [58]:
grouped_by_date.head()

,horario,item_count
0,2022-02-01,74
1,2022-02-02,88
2,2022-02-03,81
3,2022-02-04,110
4,2022-02-05,112


In [59]:
from chart_studio.grid_objs import Grid, Column
import plotly.figure_factory as ff

# Create a plot.ly-friendly table for Sales per day

table = ff.create_table(grouped_by_date)
py.iplot(table, filename='Vendas_por_dia_farina.html')



In [60]:
vendas = px.line(grouped_by_date, x = 'horario', y = 'item_count', title = 'Itens vendidos por dia', labels= {'item_count' : 'Quantidade de itens vendidos','data': 'Data'})
py.iplot(vendas, filename='Vendas_line_chart.html')

In [61]:
grouped_by_date.head()


,horario,item_count
0,2022-02-01,74
1,2022-02-02,88
2,2022-02-03,81
3,2022-02-04,110
4,2022-02-05,112


In [62]:
# ajustando o gráfio para retirar dois outliers (490 em 2022-07-30 , 2303 em 2022-06-23 ), utilizando-se de filtro com a função 'where'
indice_atip = np.where((grouped_by_date['item_count'] < 400))
linha=grouped_by_date.loc[indice_atip]

In [63]:
# https://towardsdatascience.com/line-chart-animation-with-plotly-on-jupyter-e19c738dc882



#linha=grouped_by_date: usar se precisar demonstrar os dois outliers

trace1 = go.Scatter(x=linha['horario'][:2],
                    y=linha['item_count'][:2],
                    mode='lines',
                    line=dict(width=1.5),
                    fill = 'tozeroy',
                   fillcolor = 'lightblue'
)
frames = [dict(data= [dict(type='scatter',
                           x=linha['horario'][:k+1],
                           y=linha['item_count'][:k+1]),
                     ],
               traces= [0],  
              )for k  in  range(1, len(linha)-1)]
layout = go.Layout(width=1180,
                   height=750,
                   showlegend=False,
                   hovermode='closest',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.06,
                            x=0.12,
                            xanchor='center',
                            yanchor='auto',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=30, 
                                                  redraw=False),
                                                  transition=dict(duration=0),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        
                    ]              
                  )
layout.update(xaxis =dict(range=['2022-02-01', '2022-10-31'], autorange=False),
              yaxis =dict(range=[0, 250], autorange=False)); # ajustar o range se precisar colocar os outliers
fig_vendas = go.Figure(data=[trace1], frames=frames, layout=layout)

fig_vendas.show()

C:\Users\ctobr\AppData\Local\Temp\ipykernel_24552\1892540512.py:7: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

C:\Users\ctobr\AppData\Local\Temp\ipykernel_24552\1892540512.py:8: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

C:\Users\ctobr\AppData\Local\Temp\ipykernel_24552\1892540512.py:15: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]`

# Investigando dados atipicos

In [65]:
# Vendas atípicas no dia 23 de Junho de 2022 entre 16:00 e 17:00

df_invest = pd.DataFrame(transactions).query("data == 'Jun 23, 2022'").groupby(by='hora')['item_count'].sum().reset_index()
df_invest


,hora,item_count
0,7,8
1,9,5
2,10,5
3,11,7
4,12,4
5,13,8
6,14,5
7,15,3
8,16,2254
9,17,3


In [66]:
dataframe_to_pdf(df_invest, "investiga.pdf", numpages=(1, 1), pagesize=(11.7, 8.3))

In [67]:
graf = px.line(df_invest, x = 'hora', y = 'item_count', labels= {'item_count':'Itens Vendidos'}, title= 'Itens vendidos no dia 23 de Junho 2022')

py.iplot(graf, filename='investiga_graf.html')


# Vendas dias de semana (DDS) vs Final de semana (FDS)

In [68]:
dds_vendas= pd.DataFrame(transactions[transactions.tipo_de_dia == 'DDS'].groupby(by='horario')['item_count'].sum())
fds_vendas= pd.DataFrame(transactions[transactions.tipo_de_dia == 'FDS'].groupby(by='horario')['item_count'].sum())

In [69]:
dds_vendas.head()

,item_count
horario,
2022-02-01,74
2022-02-02,88
2022-02-03,81
2022-02-04,110
2022-02-07,61


In [70]:
dds_vendas_filter = (dds_vendas['item_count']< 399)

In [71]:
fds_vendas_filter = (fds_vendas['item_count'] < 399)

In [72]:
print('O menor número de vendas em um dia de semana foi: \n{0}'.format(
    dds_vendas.item_count.min()))
print('O número médio de vendas em um dia de semana é: \n{0}'.format(
    dds_vendas.item_count.mean()))
print('O maior número de vendas em um dia de semana foi: \n{0}'.format(
    dds_vendas.item_count.max()))

O menor número de vendas em um dia de semana foi: 
1
O número médio de vendas em um dia de semana é: 
88.18333333333334
O maior número de vendas em um dia de semana foi: 
2303


In [73]:
print('O menor número de vendas em um dia de fim-de-semana foi: \n{0}'.format(
    fds_vendas.item_count.min()))
print('O número médio de vendas em um dia fim-de-semana é: \n{0}'.format(
    fds_vendas.item_count.mean()))
print('O maior número de vendas em um dia fim-de-semana foi: \n{0}'.format(
    fds_vendas.item_count.max()))

O menor número de vendas em um dia de fim-de-semana foi: 
65
O número médio de vendas em um dia fim-de-semana é: 
138.6164383561644
O maior número de vendas em um dia fim-de-semana foi: 
490


In [74]:
single_transaction = pd.DataFrame(transactions.groupby(by='venda')['item_count'].sum())

single_transaction = single_transaction.merge(transactions[['tipo_de_dia','horario', 'venda']]\
                                              .drop_duplicates('venda'), left_on=single_transaction.index, 
                                              right_on='venda', how='left', copy=False)

print('O número médio de itens em cada venda por tipo de dia é: \n{0}'.format(
    single_transaction.groupby(by='tipo_de_dia')['item_count'].mean()))

O número médio de itens em cada venda por tipo de dia é: 
tipo_de_dia
DDS    2.768707
FDS    2.828116
Name: item_count, dtype: float64


In [75]:
# Histogram of items sold by type of day

trace_dds = go.Histogram(
    x=dds_vendas[dds_vendas.item_count < 200].item_count,
    opacity=0.75,
    name = 'Dia de Semana'
)

trace_fds = go.Histogram(
    x=fds_vendas[fds_vendas.item_count < 399].item_count,
    opacity=0.65,
    name = 'Fim-de-semana',
    nbinsx = 16
)

data = [trace_dds, trace_fds]
layout = go.Layout( title = 'Distribuição das vendas por dias de semana/ finais de semana',
                   xaxis=dict(title='Número de Itens Vendidos'),
                   yaxis=dict(title='Frequencia de vendas'),
                   barmode='overlay'
                   )

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='tipo-de_dia-histograma')

# Faturamento por tipo de dia (DDS x FDS)

In [76]:
dds_fatu= pd.DataFrame(transactions[transactions.tipo_de_dia == 'DDS'].groupby(by='horario')['total_item'].sum())
fds_fatu= pd.DataFrame(transactions[transactions.tipo_de_dia == 'FDS'].groupby(by='horario')['total_item'].sum())

In [77]:
# Histograma

trace_dds = go.Histogram(
    x=dds_fatu[dds_fatu.total_item  < 7900].total_item,
    opacity=0.75,
    name = 'Dia de Semana'
)

trace_fds = go.Histogram(
    x=fds_fatu[fds_fatu.total_item < 7900].total_item,
    opacity=0.65,
    name = 'Fim-de-semana',
    nbinsx = 16
)

data = [trace_dds, trace_fds]
layout = go.Layout( title = 'Distribuição do faturamento por dias de semana/ finais de semana',
                   xaxis=dict(title='Faturamento por dia'),
                   yaxis=dict(title='Frequencia de faturamento diário'),
                   barmode='overlay'
                   )

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='tipo-de_dia-histograma')

# Número de itens vendidos por dia da semana

In [145]:
f_filter = (transactions['data']!='2022-06-23')&(transactions['data']!='2022-07-30')
transactions[f_filter]

transactions_filtrada = transactions[f_filter]

In [79]:
grouped_by_date_filtrada = pd.DataFrame(transactions_filtrada.groupby(by='horario')['item_count'].sum())

In [80]:
# Find the number of itens sold per day of week

agregado_vendas_por_dia = grouped_by_date_filtrada.merge(transactions[['dia_da_semana', 'horario']], left_index=True, right_on='horario',
        how= 'left', validate='1:m')



In [81]:
agregado_vendas_por_dia.head()

,item_count,dia_da_semana,horario
0,74,terça-feira,2022-02-01
1,74,terça-feira,2022-02-01
2,74,terça-feira,2022-02-01
3,74,terça-feira,2022-02-01
4,74,terça-feira,2022-02-01


In [82]:
agregado_vendas_por_dia = agregado_vendas_por_dia[~agregado_vendas_por_dia.index.duplicated(keep='first')]

dias = ['segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira','sábado','domingo']

vendas_por_dia = []

for dia in dias:
    
    df_dias= agregado_vendas_por_dia[agregado_vendas_por_dia.dia_da_semana == dia]
    
    vendas_por_dia.append(list(df_dias['item_count']))

In [83]:
traces = []

for i, j in enumerate(dias):
    
    trace = go.Box(
    y=vendas_por_dia[i],
    name = j
    )
    traces.append(trace)

data = traces

layout = go.Layout(title='Vendas por dia da semana',
                  xaxis = dict(title = 'Dia da Semana'),
                  yaxis = dict(title = 'Número de Itens Vendidos'))

fig = go.Figure(data = data, layout = layout)

py.iplot(fig, filename='Vendas_por_dia_Boxplot')

## Quantidade de itens vendidos por dia da semana e período


In [84]:
group_by_data_e_hora_do_dia = transactions_filtrada.groupby(by=['dia_da_semana','hora_do_dia'])['item_count'].sum()

In [85]:
# https://stackoverflow.com/questions/39275294/sort-by-certain-order-situation-pandas-dataframe-groupby
# #### Alterando a ordem de apresentação dos dias da semana(Seg - Dom), usando a lista 'dias' anteriormente criada

group_by_data_e_hora_do_dia = group_by_data_e_hora_do_dia.reindex(dias, level= 'dia_da_semana')

In [86]:
group_by_data_e_hora_do_dia = pd.DataFrame(group_by_data_e_hora_do_dia).reset_index()

In [87]:
group_by_data_e_hora_do_dia

,dia_da_semana,hora_do_dia,item_count
0,segunda-feira,manhã,587
1,segunda-feira,noite,1354
2,segunda-feira,tarde,548
3,terça-feira,manhã,545
4,terça-feira,noite,1359
5,terça-feira,tarde,619
6,quarta-feira,manhã,642
7,quarta-feira,noite,1403
8,quarta-feira,tarde,811
9,quinta-feira,manhã,526


In [88]:
group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].item_count

0      587
3      545
6      642
9      526
12     744
15    1512
18    2310
Name: item_count, dtype: int64

In [89]:
trace_manhã = go.Bar(
    y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'manhã'].dia_da_semana,
    name = "manhã",
    opacity=0.6,
)

trace_tarde = go.Bar(
   y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'tarde'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'tarde'].dia_da_semana,
    name = "tarde",
    opacity=0.6
)

trace_noite = go.Bar(
   y = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'noite'].item_count,
    x = group_by_data_e_hora_do_dia[group_by_data_e_hora_do_dia.hora_do_dia == 'noite'].dia_da_semana,
    name = "noite",
    opacity=0.6
)

layout = go.Layout(title='Vendas por período e dia da semana',
                  xaxis = dict(title = 'Dia da Semana'),
                  yaxis = dict(title = 'Número de Itens Vendidos'), hovermode= 'x')

data = [trace_manhã, trace_tarde, trace_noite]

fig = go.Figure(data = data, layout = layout)  

py.iplot(fig, filename='vendas-por-dia-periodo')

### Vendas por hora

In [90]:
vendas_por_hora = pd.DataFrame(
                transactions_filtrada[(transactions_filtrada.hora > 1) & 
                (transactions_filtrada.hora < 21)].groupby(by='hora')['item_count'].sum())

In [91]:
vendas_por_hora

,item_count
hora,
6,41
7,524
8,1289
9,1749
10,1817
11,1446
12,737
13,627
14,769


In [92]:
trace = go.Scatter(x=list(vendas_por_hora.index),
                   y=list(vendas_por_hora.item_count),
                   fill = 'tozeroy',
                   fillcolor = 'lightblue')

data = [trace]

layout = dict(
    title='Total de volume de vendas por hora',
    yaxis = dict( title = 'Volume de Itens Vendidos'),
    xaxis=dict( title = 'Hora',
        rangeselector=dict(
            buttons=list([
                dict(count = 12,
                     label='Manhã',
                     step='hour',
                     stepmode='backward'),
                dict(count = 8,
                     label='Tarde',
                     step='hour',
                     stepmode='backward'),
                dict(count = 4,
                    label='Noite',
                    step='hour',
                    stepmode='backward'),
                dict(step='all')
            ]),
            visible = True
        ),
        rangeslider=dict(
            visible = True
        )
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = 'vendas-por-hora')

# Apriori Algorithm, Method 1

In [146]:
# Create a list of lists that is conducive to the format expected from apyori

list_of_lists = []

for transaction in list(set(transactions_filtrada.venda)):
    
    df = transactions_filtrada[transactions_filtrada.venda == transaction]
    values = df.produto.values
    
    list_of_lists.append(list(values))

In [147]:
# imprting the 'apriori' function from the 'apyori' module
from apyori import apriori

# Create a list of association rules
association_rules = apriori(list_of_lists, min_confidence = 0, min_support = 0.005, min_length=2)  
association_results = list(association_rules)

In [148]:
def apriori_results(min_confidence=0, min_support=0.005, sort_by_value = 'lift', min_length =None,
                    hora_do_dia=None, dia_da_semana = None, df=transactions_filtrada):
    
    """
    Takes in a data frame, filters it by time of day, calculates the results of the Apriori algorithm,
    and returns a data frame sorted by lift.
    """
    
    from apyori import apriori
    
    if hora_do_dia != None:
        
        df = df[df.hora_do_dia == hora_do_dia]
        
    if dia_da_semana:
        
        df = df[df.tipo_de_dia == dia_da_semana]
        
    list_of_lists = []

    for transaction in list(set(df.venda)):

        df_t = df[df.venda == transaction]
        values = df_t.produto.values

        list_of_lists.append(list(values))

    if min_length is not None:
        
        list_of_lists = [i for i in list_of_lists if len(i) >= min_length]
        
    association_rules = apriori(list_of_lists, min_confidence = min_confidence, min_support = min_support)  
    association_results = list(association_rules)

    item_1 = []
    item_2 = []
    support = []
    confidence = []
    lift = []

    for item in association_results:

        pair = item[0] 
        items = [x for x in pair]
        
        if min_length is not None:
            
            if len(items) < min_length:

                continue

            else:

                item_1.append(items[0])

            #try:

                item_2.append(items[1])

            #except:

            #    item_2.append('None')
            
        else:
            
            item_1.append(items[0])

            try:

                item_2.append(items[1])

            except:

                item_2.append('None')

        support.append(item[1])

        confidence.append(item[2][0][2])
        lift.append(item[2][0][3])

    df_apriori = pd.DataFrame({'item_1':item_1, 'item_2':item_2, 'support':support,'confidence':confidence,'lift':lift})
    
    df_apriori = df_apriori.sort_values(by=sort_by_value, ascending=False)
    
    return df_apriori

In [149]:
all_day_results = apriori_results()

In [150]:
all_day_results

,item_1,item_2,support,confidence,lift
0,ADICIONAL DE MUSSARELA,None,0.011277,0.011277,1.0
103,CROISSANT,BOWL DE IOGURTE FRUTAS AVEIA,0.008105,0.008105,1.0
105,CROISSANT,CAFE CATUAI VERMELHO C/LEITE G,0.007870,0.007870,1.0
106,CROISSANT,CAFE CATUAI VERMELHO G 200ML,0.013744,0.013744,1.0
107,PASTEL DE NATA 60G,CAFE CATUAI VERMELHO G 200ML,0.005873,0.005873,1.0
108,CROISSANT,CAFE COLMEIA C/ LEITE G 200ML,0.005521,0.005521,1.0
109,CROISSANT,CAFE COLMEIA G 200ML,0.007635,0.007635,1.0
110,CAFE EQUILIBRADO CAPSULA G,CROISSANT,0.005169,0.005169,1.0
111,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.009045,0.009045,1.0
112,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.010572,0.010572,1.0


In [152]:
morning_results = apriori_results(hora_do_dia ='manhã')

In [153]:
morning_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI HORTELA 400 ML,None,0.006217,0.006217,1.0
152,CROISSANT,COOKIES 80G,0.016874,0.016874,1.0
141,PAO SOURDOUGH OVOS,CAFE EQUILIBRADO CAPSULA G,0.007105,0.007105,1.0
142,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.011545,0.011545,1.0
143,CAFE EQUILIBRADO CAPSULA P,CROISSANT OVOS BACON MEL,0.005773,0.005773,1.0
144,PAO SOURDOUGH OVOS,CAFE EQUILIBRADO CAPSULA P,0.005329,0.005329,1.0
145,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.008437,0.008437,1.0
146,CROISSANT,CAFE INTENSO CAPSULA G 150ML,0.009325,0.009325,1.0
147,PAO SOURDOUGH OVOS,CAFE INTENSO CAPSULA G 150ML,0.005773,0.005773,1.0
148,CAFE INTENSO CAPSULA P 50ML,CROISSANT,0.007993,0.007993,1.0


In [101]:
afternoon_results = apriori_results(hora_do_dia ='tarde')

In [135]:
afternoon_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.011337,0.011337,1.0
1,ADICIONAL DE MUSSARELA,None,0.010392,0.010392,1.0
102,CAFE EQUILIBRADO CAPSULA G,PASTEL DE NATA 60G,0.005668,0.005668,1.0
103,CAFE EQUILIBRADO CAPSULA P,CAFE INTENSO CAPSULA P 50ML,0.007085,0.007085,1.0
104,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.007558,0.007558,1.0
105,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.014643,0.014643,1.0
106,CAFE INTENSO CAPSULA P 50ML,CROISSANT,0.005668,0.005668,1.0
107,CAFE INTENSO CAPSULA P 50ML,PASTEL DE NATA 60G,0.007085,0.007085,1.0
108,COCA COLA LATA 350 ML,CROISSANT,0.011809,0.011809,1.0
109,COCA COLA LATA 350 ML,FOLHADO FRANGO 160G,0.005196,0.005196,1.0


In [103]:
evening_results = apriori_results(hora_do_dia='noite')

In [136]:
evening_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.005549,0.005549,1.0
122,CROISSANT,FOLHADO FRANGO 160G,0.012545,0.012545,1.0
104,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.009650,0.009650,1.0
105,COCA COLA LATA 350 ML,CROISSANT,0.012063,0.012063,1.0
106,CROISSANT BRIE PARMA MEL DE,COCA COLA LATA 350 ML,0.005549,0.005549,1.0
107,COCA COLA LATA 350 ML,FOLHADO FRANGO 160G,0.005790,0.005790,1.0
108,COCA COLA LATA 350 ML,FOLHADO MACA 80G,0.007238,0.007238,1.0
109,COCA COLA LATA 350 ML,PASTEL DE NATA 60G,0.009168,0.009168,1.0
110,COCA COLA LATA 350 ML,PIZZA AL TAGLIO GORGONZOLA E,0.006996,0.006996,1.0
111,COCA COLA LATA 350 ML,SANDUICHE DE PEPERONI BRIE,0.005066,0.005066,1.0


In [105]:
weekday_results = apriori_results(dia_da_semana= 'DDS')

In [137]:
weekday_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.007812,0.007812,1.0
94,CROISSANT,BOMBOLONI DOCE DE LEITE 80G,0.007422,0.007422,1.0
96,BOMBOLONI NUTELLA 80G,CROISSANT,0.009570,0.009570,1.0
97,BOMBOLONI NUTELLA 80G,FOLHADO FRANGO 160G,0.005469,0.005469,1.0
98,BOMBOLONI NUTELLA 80G,PASTEL DE NATA 60G,0.007227,0.007227,1.0
99,PAO SOURDOUGH OVOS,BOWL DE IOGURTE FRUTAS AVEIA,0.006836,0.006836,1.0
100,CROISSANT,CAFE CATUAI VERMELHO C/LEITE G,0.006055,0.006055,1.0
101,CROISSANT,CAFE CATUAI VERMELHO G 200ML,0.011914,0.011914,1.0
102,CROISSANT,CAFE COLMEIA C/ LEITE G 200ML,0.005859,0.005859,1.0
103,CROISSANT,CAFE COLMEIA G 200ML,0.008789,0.008789,1.0


In [107]:
weekend_results = apriori_results(dia_da_semana = 'FDS')

In [138]:
weekend_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.008247,0.008247,1.0
131,CROISSANT AMENDOAS 100G,COOKIES 80G,0.006186,0.006186,1.0
122,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.011193,0.011193,1.0
123,CAFE EQUILIBRADO CAPSULA P,CROISSANT AMENDOAS 100G,0.005302,0.005302,1.0
124,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.011487,0.011487,1.0
125,CAFE INTENSO CAPSULA P 50ML,CROISSANT,0.005007,0.005007,1.0
126,CAFE INTENSO CAPSULA P 50ML,PASTEL DE NATA 60G,0.005007,0.005007,1.0
127,COCA COLA LATA 350 ML,CROISSANT,0.010015,0.010015,1.0
128,COCA COLA LATA 350 ML,PASTEL DE NATA 60G,0.007069,0.007069,1.0
129,CROISSANT,COCA COLA ZERO LATA 350 ML,0.008542,0.008542,1.0


In [109]:
weekday_morning_results = apriori_results(dia_da_semana = 'DDS', hora_do_dia = 'manhã')

In [110]:
weekday_morning_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.009506,0.009506,1.0
148,CAFE INTENSO CAPSULA P 50ML,CROISSANT OVOS BACON MEL,0.005703,0.005703,1.0
137,PAO SOURDOUGH OVOS,CAFE EQUILIBRADO CAPSULA G,0.009506,0.009506,1.0
138,CAFE EQUILIBRADO CAPSULA G,PASTEL DE NATA 60G,0.005703,0.005703,1.0
139,CAFE EQUILIBRADO CAPSULA P,CAFE INTENSO CAPSULA P 50ML,0.005703,0.005703,1.0
140,CAFE EQUILIBRADO CAPSULA P,CROISSANT,0.014259,0.014259,1.0
141,CAFE EQUILIBRADO CAPSULA P,CROISSANT OVOS BACON MEL,0.007605,0.007605,1.0
142,CAFE EQUILIBRADO CAPSULA P,FOLHADO FRANGO 160G,0.005703,0.005703,1.0
143,CAFE EQUILIBRADO CAPSULA P,PASTEL DE NATA 60G,0.009506,0.009506,1.0
144,CROISSANT,CAFE INTENSO CAPSULA G 150ML,0.013308,0.013308,1.0


In [111]:
weekday_afternoon_results = apriori_results(dia_da_semana = 'DDS', hora_do_dia = 'tarde')

In [112]:
weekday_afternoon_results[:20]

,item_1,item_2,support,confidence,lift
0,ABACAXI CURCUMA GENGIBRE,None,0.011021,0.011021,1.0
119,FOLHADO FRANGO 160G,COOKIES 80G,0.005143,0.005143,1.0
111,COCA COLA LATA 350 ML,CROISSANT,0.011756,0.011756,1.0
112,COCA COLA LATA 350 ML,FOLHADO FRANGO 160G,0.005143,0.005143,1.0
113,COCA COLA LATA 350 ML,PASTEL DE NATA 60G,0.005878,0.005878,1.0
114,COCA COLA LATA 350 ML,SANDUICHE DE PEPERONI BRIE,0.007348,0.007348,1.0
115,CROISSANT,COOKIES 80G,0.016899,0.016899,1.0
116,CROISSANT AMENDOAS 100G,COOKIES 80G,0.005143,0.005143,1.0
117,CROISSANT TRADICIONAL 80 G,COOKIES 80G,0.005143,0.005143,1.0
118,FOCACCIA FORMAGGIO 246G,COOKIES 80G,0.006613,0.006613,1.0


# Apriori Algorithm, Method 2

In [113]:
def apriori_results_method_2(df = transactions_filtrada, min_support = 0.005, hora_do_dia = None, dia_da_semana = None,
                            min_threshold = 1, metric = 'lift'):
    
    """
    Takes in a data frame and returns the results of the Apriori algorithm using the mlxtend Python package.
    """
    
    from mlxtend.frequent_patterns import apriori
    from mlxtend.frequent_patterns import association_rules

    if hora_do_dia:
        
        df = df[df.hora_do_dia == hora_do_dia]
        
    if dia_da_semana:
        
        df = df[df.tipo_de_dia == dia_da_semana]
        
    dummies_df = df.groupby(['venda', 'produto'])['produto'].count().unstack().\
                 reset_index().fillna(0).set_index('venda')
    
    def encode_units(x):
    
        if x <= 0:
            return 0
        if x >= 1:
            return 1
    
    dummies_df = dummies_df.applymap(encode_units)
    
    frequent_itemsets = apriori(dummies_df, min_support= min_support, use_colnames=True)

    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)
    
    rules.drop_duplicates(subset='leverage', keep='first', inplace=True)
    rules = rules.sort_values(by='lift', ascending=False)
    
    return rules

In [114]:
all_day_method_2 = apriori_results_method_2()

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [115]:
morning_method_2 = apriori_results_method_2(hora_do_dia = 'manhã')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [139]:
morning_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
217,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.013316,0.016423,0.009321,0.700000,42.624324,0.009102,3.278592
216,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.015091,0.014647,0.009321,0.617647,42.168449,0.009100,2.577077
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.016423,0.014647,0.009765,0.594595,40.594595,0.009524,2.430537
132,(COOKIES 80G IFOOD),(CROISSANT IFOOD),0.009321,0.081225,0.005770,0.619048,7.621390,0.005013,2.411784
176,(CROISSANT IFOOD),(SOURDOUGH 369G IFOOD),0.081225,0.014203,0.008433,0.103825,7.309939,0.007280,1.100005
174,(PASTEL DE NATA 60G IFOOD),(CROISSANT IFOOD),0.015535,0.081225,0.008433,0.542857,6.683372,0.007171,2.009820
178,(CROISSANT MUSSATRELA DE),(SUCO DE LARANJA 400ML),0.027075,0.036840,0.006658,0.245902,6.674896,0.005660,1.277234
170,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.039947,0.036840,0.008877,0.222222,6.032129,0.007405,1.238349
196,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.050599,0.024856,0.007102,0.140351,5.646617,0.005844,1.134351
246,"(PAO SOURDOUGH OVOS, CROISSANT)",(CAFE CATUAI VERMELHO G 200ML),0.017310,0.055038,0.005326,0.307692,5.590571,0.004374,1.364946


In [117]:
afternoon_method_2 = apriori_results_method_2(hora_do_dia = 'tarde')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [118]:
afternoon_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
118,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.008975,0.007085,0.006613,0.736842,103.992982,0.006550,3.773075
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.010392,0.007085,0.007085,0.681818,96.227273,0.007012,3.120588
119,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.006613,0.010392,0.006613,1.000000,96.227273,0.006544,inf
84,(PASTEL DE NATA 60G IFOOD),(CROISSANT IFOOD),0.011809,0.045820,0.005196,0.440000,9.602887,0.004655,1.703894
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.168162,0.007085,0.006613,0.039326,5.550187,0.005422,1.033560
32,(CAFE EQUILIBRADO CAPSULA P),(CAFE INTENSO CAPSULA P 50ML),0.042985,0.030704,0.007085,0.164835,5.368555,0.005766,1.160605
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.168162,0.010392,0.008975,0.053371,5.135725,0.007227,1.045402
96,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.053377,0.032121,0.008030,0.150442,4.683628,0.006316,1.139274
12,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.048181,0.059046,0.012754,0.264706,4.483059,0.009909,1.279698
64,(CROISSANT),(LARANJA 400 ML),0.168162,0.028342,0.017005,0.101124,3.567978,0.012239,1.080970


In [119]:
evening_method_2 = apriori_results_method_2(hora_do_dia = 'noite')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [120]:
evening_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
92,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.055247,0.024367,0.006273,0.113537,4.659518,0.004926,1.100591
0,(CROISSANT),(ADICIONAL DE MUSSARELA),0.188661,0.008926,0.006514,0.034527,3.867941,0.004830,1.026516
12,(CROISSANT BRIE PARMA MEL DE),(AGUA MINERAL SEM GAS 500 ML),0.055247,0.040290,0.007479,0.135371,3.359961,0.005253,1.109968
14,(CROISSANT MUSSATRELA DE),(AGUA MINERAL SEM GAS 500 ML),0.038601,0.040290,0.005066,0.131250,3.257672,0.003511,1.104703
74,(CROISSANT),(LARANJA 400 ML),0.188661,0.020024,0.011821,0.062660,3.129218,0.008044,1.045486
88,(CROISSANT BRIE PARMA MEL DE),(CROISSANT MUSSATRELA DE),0.055247,0.038601,0.006514,0.117904,3.054449,0.004381,1.089903
4,(BOMBOLONI NUTELLA 80G),(AGUA MINERAL S/GAS 500ML),0.046080,0.042461,0.005549,0.120419,2.836001,0.003592,1.088631
128,(PIZZA AL TAGLIO PEPPERONI 210G),(PIZZA AL TAGLIO GORGONZOLA E),0.034499,0.055247,0.005066,0.146853,2.658106,0.003160,1.107374
18,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.046080,0.056936,0.006755,0.146597,2.574763,0.004132,1.105063
50,(COCA COLA LATA 350 ML),(PIZZA AL TAGLIO GORGONZOLA E),0.049940,0.055247,0.006996,0.140097,2.535810,0.004237,1.098673


In [121]:
weekend_method_2 = apriori_results_method_2(dia_da_semana = 'FDS')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [122]:
weekend_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
174,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009426,0.006480,0.005007,0.531250,81.981534,0.004946,2.119509
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.010604,0.006480,0.005007,0.472222,72.872475,0.004939,1.882459
130,(PASTEL DE NATA 60G IFOOD),(CROISSANT IFOOD),0.012077,0.064507,0.006186,0.512195,7.940194,0.005407,1.917761
50,(PAO SOURDOUGH OVOS),(BOWL DE IOGURTE FRUTAS AVEIA),0.032695,0.043004,0.009426,0.288288,6.703690,0.008020,1.344639
124,(CROISSANT BRIE PARMA MEL DE),(SUCO DE LARANJA 400ML),0.047717,0.027982,0.007658,0.160494,5.735543,0.006323,1.157845
152,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.045066,0.022975,0.005302,0.117647,5.120664,0.004267,1.107295
134,(CROISSANT MUSSATRELA DE),(SUCO DE LARANJA 400ML),0.036230,0.027982,0.005007,0.138211,4.939238,0.003994,1.127907
68,(CAFE INTENSO CAPSULA P 50ML),(CAFE EQUILIBRADO CAPSULA P),0.022091,0.047717,0.005007,0.226667,4.750206,0.003953,1.231400
176,"(ADICIONAL DE MUSSARELA, ADICIONAL DE PRESUNTO)",(CROISSANT),0.005007,0.233284,0.005007,1.000000,4.286616,0.003839,inf
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.233284,0.006480,0.006480,0.027778,4.286616,0.004968,1.021906


In [123]:
weekday_method_2 = apriori_results_method_2(dia_da_semana = 'DDS')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [124]:
weekday_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
102,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.009375,0.008398,0.006055,0.645833,76.899225,0.005976,2.799816
103,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.007227,0.011719,0.006055,0.837838,71.495495,0.005970,6.094401
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.011719,0.008398,0.006836,0.583333,69.457364,0.006738,2.379844
26,(PAO SOURDOUGH OVOS),(BOWL DE IOGURTE FRUTAS AVEIA),0.023633,0.024219,0.006836,0.289256,11.943482,0.006264,1.372902
86,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.047852,0.022852,0.005469,0.114286,5.001221,0.004375,1.103232
104,"(ADICIONAL DE MUSSARELA, ADICIONAL DE PRESUNTO)",(CROISSANT),0.006836,0.188477,0.006055,0.885714,4.699334,0.004766,7.100830
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.188477,0.008398,0.007227,0.038342,4.565369,0.005644,1.031137
12,(CROISSANT BRIE PARMA MEL DE),(AGUA MINERAL SEM GAS 500 ML),0.043945,0.033594,0.006641,0.151111,4.498191,0.005164,1.138437
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.188477,0.011719,0.009375,0.049741,4.244560,0.007166,1.040012
16,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.041016,0.046484,0.007227,0.176190,3.790316,0.005320,1.157447


In [125]:
weekend_morning_method_2 = apriori_results_method_2(dia_da_semana = 'FDS', hora_do_dia = 'manhã')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [126]:
weekend_morning_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
266,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.010824,0.009992,0.005828,0.538462,53.891026,0.005720,2.145018
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.011657,0.009992,0.005828,0.500000,50.041667,0.005712,1.980017
142,(CIABATTA 123G),(OVOS COM BACON E BRIE),0.019983,0.024147,0.006661,0.333333,13.804598,0.006179,1.463780
220,(CROISSANT OVOS),(SUCO DE LARANJA 400ML),0.017485,0.044963,0.007494,0.428571,9.531746,0.006708,1.671316
285,"(PASTEL DE NATA 60G, AGUA MINERAL S/GAS 500ML)",(SANDUICHE DE PEPERONI BRIE),0.009159,0.067444,0.005828,0.636364,9.435466,0.005211,2.564530
64,(BOMBOLONI NUTELLA 80G IFOOD),(CROISSANT IFOOD),0.006661,0.098251,0.005828,0.875000,8.905720,0.005174,7.213988
128,(CAFE EQUILIBRADO CAPSULA G),(CROISSANT OVOS),0.043297,0.017485,0.005828,0.134615,7.698718,0.005071,1.135350
40,(CROISSANT OVOS),(AGUA MINERAL SEM GAS 500 ML),0.017485,0.044963,0.005828,0.333333,7.413580,0.005042,1.432556
214,(CROISSANT IFOOD),(SOURDOUGH 369G IFOOD),0.098251,0.015820,0.010824,0.110169,6.963872,0.009270,1.106031
238,(MINI CROISSANT 25G KIT C 5),(MINI CROISSANT PAIN DE CHOC),0.062448,0.024979,0.009992,0.160000,6.405333,0.008432,1.160739


In [127]:
weekday_afternoon_method_2 = apriori_results_method_2(dia_da_semana = 'DDS', hora_do_dia = 'tarde')

c:\Users\ctobr\miniconda3\envs\minimal_ds\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



In [128]:
weekday_afternoon_method_2[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
146,"(CROISSANT, ADICIONAL DE MUSSARELA)",(ADICIONAL DE PRESUNTO),0.006613,0.005878,0.005143,0.777778,132.319444,0.005104,4.473549
147,"(CROISSANT, ADICIONAL DE PRESUNTO)",(ADICIONAL DE MUSSARELA),0.005143,0.008082,0.005143,1.000000,123.727273,0.005102,inf
0,(ADICIONAL DE MUSSARELA),(ADICIONAL DE PRESUNTO),0.008082,0.005878,0.005878,0.727273,123.727273,0.005831,3.645114
114,(PASTEL DE NATA 60G IFOOD),(CROISSANT IFOOD),0.014695,0.033064,0.006613,0.450000,13.610000,0.006127,1.758066
154,"(BOMBOLONI NUTELLA 80G, PASTEL DE NATA 60G)",(FOLHADO FRANGO 160G),0.008082,0.053637,0.005143,0.636364,11.864259,0.004710,2.602498
152,"(FOLHADO FRANGO 160G, PASTEL DE NATA 60G)",(BOMBOLONI NUTELLA 80G),0.013226,0.046289,0.005143,0.388889,8.401235,0.004531,1.560617
4,(CROISSANT),(ADICIONAL DE PRESUNTO),0.161646,0.005878,0.005143,0.031818,5.413068,0.004193,1.026793
42,(CAFE EQUILIBRADO CAPSULA P),(CAFE INTENSO CAPSULA P 50ML),0.043350,0.032329,0.007348,0.169492,5.242681,0.005946,1.165155
2,(CROISSANT),(ADICIONAL DE MUSSARELA),0.161646,0.008082,0.006613,0.040909,5.061570,0.005306,1.034227
16,(BOMBOLONI NUTELLA 80G),(BOMBOLONI DOCE DE LEITE 80G),0.046289,0.050698,0.011756,0.253968,5.009432,0.009409,1.272469


In [129]:
weekday_afternoon_method_2.groupby(by='antecedents')['lift'].sum().sort_values(ascending=False)

antecedents
(CROISSANT, ADICIONAL DE MUSSARELA)             132.319444
(CROISSANT, ADICIONAL DE PRESUNTO)              123.727273
(ADICIONAL DE MUSSARELA)                        123.727273
(CROISSANT)                                      36.009148
(COCA COLA LATA 350 ML)                          14.640432
(PASTEL DE NATA 60G IFOOD)                       13.610000
(PASTEL DE NATA 60G)                             12.214096
(BOMBOLONI NUTELLA 80G, PASTEL DE NATA 60G)      11.864259
(FOLHADO FRANGO 160G, PASTEL DE NATA 60G)        10.807043
(BOMBOLONI NUTELLA 80G)                          10.075422
(CROISSANT  TRADICIONAL 80 G)                     9.120090
(CAFE EQUILIBRADO CAPSULA P)                      9.007430
(FOLHADO FRANGO 160G)                             8.866526
(SOURDOUGH AZEITONA E ALECRIM)                    8.297032
(MINI CROISSANT 25G KIT C 5)                      7.845827
(CROISSANT  AMENDOAS 100G)                        5.951058
(SOURDOUGH 369G)                            